In [41]:
import argparse
import logging
import math
import os
import random
from collections import namedtuple
from typing import Optional, Union

import matplotlib.pyplot as plt
import numpy as np
import torch
import tqdm
from torch import distributions, nn, optim

import torchsde

In [42]:
# check if the gpu is available or not, if yes, use gpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [55]:
args = {
    "likelihood": "laplace", # specify the likelihood distribution,p(x|z)
    "adjoint": True, # specify whether to use adjoint sensitivty method to backward the sde
    "debug": True, # specify whether to debug or not
    "data": "segmented_cosine",# specify which data to use
    "kl_anneal_iters": 100, # number of the iterations of the annealing kl divergence schedule
    "train_iters": 500, # number of iterations to train the model
    "batch_size": 100, 
    "adaptive": False, # whether use adaptive solver or not
    "method": "euler", # the method of sde solver
    "dt": 1e-2, # the parameter dt of sde solver
    "rtol": 1e-3, # the parameter rtol of sde solver
    "atol": 1e-3, # the atol of sde solver
    "scale": 0.05, # the scale, of the likelihood distribution
    "dpi": 500, # dpi of images
    "pause_iters": 50 # the interval to evaluate the model
}

In [44]:
# w/ underscore -> numpy; w/o underscore -> torch.
'''
    ts: original time series; can be segmented or irregular
    ts_ext: with extended time outisde the time series, use to generate latent outside to penalize out-of-data region and spread uncertainty
    ts_vis: regular time series used to plot the data
    ys: the observed dynamic, same size as ts
'''

Data = namedtuple('Data', ['ts_', 'ts_ext_', 'ts_vis_', 'ts', 'ts_ext', 'ts_vis', 'ys', 'ys_'])

In [45]:
class LinearScheduler(object):
    '''
        output a value follows linear schedule from maxval/iters to maxval, with 'iters' steps
    '''
    
    '''
        iters = 100,
        maxval = 1,
        1/100 = 0.001, 0.002, .... 1
    '''
    
    def __init__(self, iters, maxval=1.0):
        self._iters = max(1, iters)
        self._val = maxval / self._iters
        self._maxval = maxval

    def step(self):
        self._val = min(self._maxval, self._val + self._maxval / self._iters)

    @property
    def val(self):
        return self._val

In [56]:
class EMAMetric(object):
    '''
        Exponential moving average, used to calculate the average
    '''
    def __init__(self, gamma: Optional[float] = .99):
        super(EMAMetric, self).__init__()
        self._val = 0.
        self._gamma = gamma

    def step(self, x: Union[torch.Tensor, np.ndarray]):
        x = x.detach().cpu().numpy() if torch.is_tensor(x) else x
        self._val = self._gamma * self._val + (1 - self._gamma) * x
        return self._val

    @property
    def val(self):
        return self._val

In [57]:
def manual_seed(seed: int):
    '''
        set the random seed, make sure the result keeps the same for each call
    '''
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [58]:
def _stable_division(a, b, epsilon=1e-7): #a/b
    '''
        change all elements x in b s.t -epsilon < x < epsilon to epsilon, to make sure the division is stable, won't cause a really large number
    '''
    
    b = torch.where(b.abs().detach() > epsilon, b, torch.full_like(b, fill_value=epsilon) * b.sign())
    return a / b

In [63]:
class LatentSDE(torchsde.SDEIto):

    def __init__(self, theta=1.0, mu=0.0, sigma=0.5):
        super(LatentSDE, self).__init__(noise_type="diagonal")
        logvar = math.log(sigma ** 2 / (2. * theta)) # calculate the log variance

        # Prior drift.
        self.register_buffer("theta", torch.tensor([[theta]])) # prior parameters, register 成buffer, 参数不会进行更新
        self.register_buffer("mu", torch.tensor([[mu]]))
        self.register_buffer("sigma", torch.tensor([[sigma]]))

        # p(z0).
        self.register_buffer("py0_mean", torch.tensor([[mu]])) # setup the prior distribution
        self.register_buffer("py0_logvar", torch.tensor([[logvar]]))

        # Approximate posterior drift: Takes in 2 positional encodings and the state. f(t,y)
        self.net = nn.Sequential( #h\Phi()
            nn.Linear(3, 200),
            nn.Tanh(),
            nn.Linear(200, 200),
            nn.Tanh(),
            nn.Linear(200, 1)
        )
        
        # Initialization trick from Glow. 
        self.net[-1].weight.data.fill_(0.)
        self.net[-1].bias.data.fill_(0.)

        # q(y0). the initial value of the approx posterior distribution
        self.qy0_mean = nn.Parameter(torch.tensor([[mu]]), requires_grad=True) 
        self.qy0_logvar = nn.Parameter(torch.tensor([[logvar]]), requires_grad=True)

    def f(self, t, y):  # Approximate posterior drift. f(t, y) = t # h\Phi
        if t.dim() == 0:
            t = torch.full_like(y, fill_value=t)
        # Positional encoding in transformers for time-inhomogeneous posterior.
        return self.net(torch.cat((torch.sin(t), torch.cos(t), y), dim=-1))

    def g(self, t, y):  # Shared diffusion. g(t,y) = sigma
        return self.sigma.repeat(y.size(0), 1)

    def h(self, t, y):  # Prior drift. h(t,y) = theta * (mu-y) # h\Theta
        return self.theta * (self.mu - y) # need to figure out

    def f_aug(self, t, y):  # Drift for augmented dynamics with logqp term.
        '''
             y has two columns, the first column is y0, the one we want to generate the SDE dynamic
             the second column is 0, used to generate the sampling paths from the posterior process, and used to estimate the kl divergence
        '''
        y = y[:, 0:1] # get the first column of y, that is to get y0 # z0
        f, g, h = self.f(t, y), self.g(t, y), self.h(t, y) # calculate f, g, h
        u = _stable_division(f - h, g) # u(z,t) = (f-h)/g
        f_logqp = .5 * (u ** 2).sum(dim=1, keepdim=True) # (u^2)/2, the drift of the second sde
        return torch.cat([f, f_logqp], dim=1) # [batch_size, 2]

    def g_aug(self, t, y):  # Diffusion for augmented dynamics with logqp term.
        y = y[:, 0:1]
        g = self.g(t, y)
        g_logqp = torch.zeros_like(y) # the diffusion of the second sde
        return torch.cat([g, g_logqp], dim=1) # [batch_size, 2]

    def forward(self, ts, batch_size, eps=None):
        # recognition process
        eps = torch.randn(batch_size, 1).to(self.qy0_std) if eps is None else eps
        y0 = self.qy0_mean + eps * self.qy0_std # randomly generate z0 from approx posterior distribution q(z|x)
        
        
        qy0 = distributions.Normal(loc=self.qy0_mean, scale=self.qy0_std) # approx posterior distribution
        py0 = distributions.Normal(loc=self.py0_mean, scale=self.py0_std) # prior distribution
        
        logqp0 = distributions.kl_divergence(qy0, py0).sum(dim=1)  # KL(t=0). # kl divergence when t = 0

        aug_y0 = torch.cat([y0, torch.zeros(batch_size, 1).to(y0)], dim=1)
        aug_ys = sdeint_fn(
            sde=self,
            y0=aug_y0,
            ts=ts, #[0,0.1,0.2, 0.3]
            method=args["method"],
            dt=args["dt"],
            adaptive=args["adaptive"],
            rtol=args["rtol"],
            atol=args["atol"],
            names={'drift': 'f_aug', 'diffusion': 'g_aug'}
        )
        #[len(ts),batch_size,2]
        ys, logqp_path = aug_ys[:, :, 0:1], aug_ys[-1, :, 1] 
        # the first column of the last dimension is the sample dynamic
        # the second column of the last dimension is the kl divergence
        logqp = (logqp0 + logqp_path).mean(dim=0)  # KL(t=0) + KL(path). # calculate the kl divergence
        return ys, logqp

    def sample_p(self, ts, batch_size, eps=None, bm=None):
        '''
            latent variable samples from prior distribution p(z), and their SDE dynamics
        '''
        eps = torch.randn(batch_size, 1).to(self.py0_mean) if eps is None else eps
        y0 = self.py0_mean + eps * self.py0_std # [batch_size, 1]: [1024, 1]
        
        yt = sdeint_fn(self, y0, ts, bm=bm, method='srk', dt=args["dt"], names={'drift': 'h'}) # [len(ts), batch_size, 1]: [300, 1024,1]
        return yt

    def sample_q(self, ts, batch_size, eps=None, bm=None):
        '''
            latent variable samples from approx posterior distribution q(z|x), and their SDE dynamics
        '''
        eps = torch.randn(batch_size, 1).to(self.qy0_mean) if eps is None else eps
        y0 = self.qy0_mean + eps * self.qy0_std # [batch_size, 1]: [1024, 1]
        return sdeint_fn(self, y0, ts, bm=bm, method='srk', dt=args["dt"]) # [len(ts), batch_size, 1]: [300, 1024, 1]

    # self.py_std = 
    @property # declare it as a property, then we can access it through self.py0_std and specify setter and getter
    def py0_std(self): # the standard deviation of the prior distribution p(z)
        return torch.exp(.5 * self.py0_logvar)

    @property
    def qy0_std(self): # the standard deviation of the approx posterior distribution q(z|x)
        return torch.exp(.5 * self.qy0_logvar)



In [64]:
def make_segmented_cosine_data():
    ts_ = np.concatenate((np.linspace(0.3, 0.8, 10), np.linspace(1.2, 1.5, 10)), axis=0) # create segmented time series
    ts_ext_ = np.array([0.] + list(ts_) + [2.0]) # add out-of-data time point
    ts_vis_ = np.linspace(0., 2.0, 300) # regular time series used for visualization
    ys_ = np.cos(ts_ * (2. * math.pi))[:, None] # get the segmented cosine data

    ts = torch.tensor(ts_).float()
    ts_ext = torch.tensor(ts_ext_).float()
    ts_vis = torch.tensor(ts_vis_).float()
    ys = torch.tensor(ys_).float().to(device)
    return Data(ts_, ts_ext_, ts_vis_, ts, ts_ext, ts_vis, ys, ys_)


def make_irregular_sine_data():
    ts_ = np.sort(np.random.uniform(low=0.4, high=1.6, size=16)) # create irregular time series
    ts_ext_ = np.array([0.] + list(ts_) + [2.0]) # add out-of-data time point
    ts_vis_ = np.linspace(0., 2.0, 300) # regular time series used for visualization
    ys_ = np.sin(ts_ * (2. * math.pi))[:, None] * 0.8 # get the irregular sine data

    ts = torch.tensor(ts_).float()
    ts_ext = torch.tensor(ts_ext_).float()
    ts_vis = torch.tensor(ts_vis_).float()
    ys = torch.tensor(ys_).float().to(device)
    return Data(ts_, ts_ext_, ts_vis_, ts, ts_ext, ts_vis, ys, ys_)


def make_data():
    data_constructor = {
        'segmented_cosine': make_segmented_cosine_data,
        'irregular_sine': make_irregular_sine_data
    }[args["data"]]
    return data_constructor()

In [83]:
def main():
    # Dataset.
    ts_, ts_ext_, ts_vis_, ts, ts_ext, ts_vis, ys, ys_ = make_data()

    # Plotting parameters.
    vis_batch_size = 1024 # the batch_size used to visaulize
    ylims = (-1.2, 1.2) # set up the ylim of the figure
    
    alphas = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55]
    percentiles = [0.999, 0.99, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
    
    vis_idx = np.random.permutation(vis_batch_size) # shuffle the numbers from 1-vis_batch_size
    # From https://colorbrewer2.org/.

    sample_colors = ('#fc4e2a', '#e31a1c', '#bd0026') 
    fill_color = '#fd8d3c'
    
    mean_color = '#800026'
    
    num_samples = len(sample_colors)
    
    eps = torch.randn(vis_batch_size, 1).to(device)  # samples from normal distribution
    
    bm = torchsde.BrownianInterval(
        t0=ts_vis[0],
        t1=ts_vis[-1],
        size=(vis_batch_size, 1),
        device=device,
        levy_area_approximation='space-time'
    )  # We need space-time Levy area to use the SRK solver, fix the brownian motion allows us to generate the sde dynamic fexedly

    # Model.
    model = LatentSDE().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-2)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=.999)
    kl_scheduler = LinearScheduler(iters=args["kl_anneal_iters"])

    logpy_metric = EMAMetric()
    kl_metric = EMAMetric()
    loss_metric = EMAMetric()
    
    with torch.no_grad():
        zs = model.sample_p(ts=ts_vis, batch_size=vis_batch_size, eps=eps, bm=bm).squeeze() # sde dynamic sampled from prior distribution
        ts_vis_, zs_ = ts_vis.cpu().numpy(), zs.cpu().numpy()  # convert them to numpy
        # print(zs.size()) # [len(ts_vis), batch_size]: [300, 1024]
        zs_ = np.sort(zs_, axis=1) # sort each row

        img_dir = os.path.join('./img/' 'prior.png')
        plt.subplot(frameon=False)
        for alpha, percentile in zip(alphas, percentiles):
            idx = int((1 - percentile) / 2. * vis_batch_size) # 选择要考虑百分之多少的数据
            zs_bot_ = zs_[:, idx] # 计算底线
            zs_top_ = zs_[:, -idx] # 计算顶线
            plt.fill_between(ts_vis_, zs_bot_, zs_top_, alpha=alpha, color=fill_color) # 用来填充两条曲线之间的区域

        # `zorder` determines who's on top; the larger the more at the top.
        plt.scatter(ts_, ys_, marker='x', zorder=3, color='k', s=35)  # scatter plot the original observed dynamic
        plt.ylim(ylims) 
        plt.xlabel('$t$')
        plt.ylabel('$Y_t$')
        plt.tight_layout()
        plt.savefig(img_dir, dpi=args["dpi"])
        plt.close()
        logging.info(f'Saved prior figure at: {img_dir}')
    
    for global_step in tqdm.tqdm(range(args["train_iters"])):
        # Plot and save.
        
        if global_step % args["pause_iters"] == 0:
            img_path = os.path.join('./img/', f'global_step_{global_step}.png')

            with torch.no_grad():
                zs = model.sample_q(ts=ts_vis, batch_size=vis_batch_size, eps=eps, bm=bm).squeeze()
                samples = zs[:, vis_idx] # samples是按照之前的permutation来排序的
                ts_vis_, zs_, samples_ = ts_vis.cpu().numpy(), zs.cpu().numpy(), samples.cpu().numpy()
                zs_ = np.sort(zs_, axis=1)
                plt.subplot(frameon=False)
                
                
                
                # same as above, plot the percentiles
                if True: #args.show_percentiles:
                    for alpha, percentile in zip(alphas, percentiles):
                        idx = int((1 - percentile) / 2. * vis_batch_size)
                        zs_bot_, zs_top_ = zs_[:, idx], zs_[:, -idx]
                        plt.fill_between(ts_vis_, zs_bot_, zs_top_, alpha=alpha, color=fill_color)
                
                # plot the mean of all the SDE dynamics
                if True: #args.show_mean:
                    plt.plot(ts_vis_, zs_.mean(axis=1), color=mean_color)
                # plot the first three SDE dynamics, since we shuffle the samples already, so the first SDE dynamics are random
                if True: #args.show_samples:
                    for j in range(num_samples):
                        plt.plot(ts_vis_, samples_[:, j], color=sample_colors[j], linewidth=1.0)
                
                if True: #args.show_arrows:
                    num, dt = 12, 0.12
                    t, y = torch.meshgrid(
                        [torch.linspace(0.2, 1.8, num).to(device), torch.linspace(-1.5, 1.5, num).to(device)]
                    )
                    #print(t.size()) # [12,12]
                    #print(y.size()) # [12, 12]
                    t, y = t.reshape(-1, 1), y.reshape(-1, 1)
                    '''
                        ex:
                        t = [[1],[1],[2],[2],[3],[3]]
                        y = [[1],[2],[3],[1],[2],[3]]
                    '''
                    fty = model.f(t=t, y=y).reshape(num, num) # call f(t,y)
                    dt = torch.zeros(num, num).fill_(dt).to(device)
                    dy = fty * dt # calculate the gradients
                    dt_, dy_, t_, y_ = dt.cpu().numpy(), dy.cpu().numpy(), t.cpu().numpy(), y.cpu().numpy()
                    plt.quiver(t_, y_, dt_, dy_, alpha=0.3, edgecolors='k', width=0.0035, scale=50) #画箭头，画风场

                if False: #args.hide_ticks:
                    plt.xticks([], [])
                    plt.yticks([], [])

                plt.scatter(ts_, ys_, marker='x', zorder=3, color='k', s=35)  # scatter plot the Data.
                
                
                
                zs = model.sample_q(ts=ts, batch_size=args["batch_size"]).squeeze()
                likelihood_constructor = {"laplace": distributions.Laplace, "normal": distributions.Normal}[args["likelihood"]]
                likelihood = likelihood_constructor(loc=zs, scale=args["scale"]) #f(x) = p(x|z)
        
                samples = likelihood.sample()
                samples = samples.mean(axis=1)
                plt.plot(ts_,samples)
                
                plt.ylim(ylims)
                plt.xlabel('$t$')
                plt.ylabel('$Y_t$')
                plt.tight_layout()
                plt.savefig(img_path, dpi=args["dpi"])
                plt.close()
                logging.info(f'Saved figure at: {img_path}')
    
        
        # Train.
        optimizer.zero_grad()
        zs, kl = model(ts=ts_ext, batch_size=args["batch_size"]) 
        # print(zs.size()) # [len(ts_ext),batch_size,1]: [22,100,1] 
        zs = zs.squeeze() # [len(ts_ext), batch_size]: [22, 100]
        zs = zs[1:-1]  # Drop first and last which are only used to penalize out-of-data region and spread uncertainty.

        # select the likelihood function p(x|z)
        # generation process
        likelihood_constructor = {"laplace": distributions.Laplace, "normal": distributions.Normal}[args["likelihood"]]
        likelihood = likelihood_constructor(loc=zs, scale=args["scale"]) #f(x) = p(x|z)
        
        logpy = likelihood.log_prob(ys).sum(dim=0).mean(dim=0) # calculate the log likelihood p(x|z)

        loss = -logpy + kl * kl_scheduler.val # we want to maximize the ELBO
        loss.backward()

        optimizer.step()
        scheduler.step()
        kl_scheduler.step()

        logpy_metric.step(logpy)
        kl_metric.step(kl)
        loss_metric.step(loss)

        logging.info(
            f'global_step: {global_step}, '
            f'logpy: {logpy_metric.val:.3f}, '
            f'kl: {kl_metric.val:.3f}, '
            f'loss: {loss_metric.val:.3f}'
        )
    torch.save(
        {'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'kl_scheduler': kl_scheduler},
        os.path.join('./', f'global_step_{global_step}.ckpt')
    )

In [84]:
manual_seed(0)

if args["debug"]:
    logging.getLogger().setLevel(logging.INFO)

ckpt_dir = os.path.join('./', 'ckpts')
os.makedirs(ckpt_dir, exist_ok=True)

sdeint_fn = torchsde.sdeint_adjoint if args["adjoint"] else torchsde.sdeint

main()

INFO:root:Saved prior figure at: ./img/prior.png
  0%|          | 0/500 [00:00<?, ?it/s]INFO:root:Saved figure at: ./img/global_step_0.png
INFO:root:global_step: 0, logpy: -2.300, kl: 0.013, loss: 2.300
 10%|█         | 50/500 [00:52<07:04,  1.06it/s]INFO:root:Saved figure at: ./img/global_step_50.png
INFO:root:global_step: 50, logpy: -45.006, kl: 4.566, loss: 46.349
 14%|█▍        | 69/500 [01:15<06:54,  1.04it/s]INFO:root:global_step: 69, logpy: -54.620, kl: 5.429, loss: 56.744


 20%|██        | 100/500 [01:44<06:13,  1.07it/s]INFO:root:Saved figure at: ./img/global_step_100.png
INFO:root:global_step: 100, logpy: -64.293, kl: 6.613, loss: 68.163
 28%|██▊       | 139/500 [02:26<05:36,  1.07it/s]INFO:root:global_step: 139, logpy: -56.660, kl: 11.574, loss: 66.381


 30%|███       | 150/500 [02:36<05:27,  1.07it/s]INFO:root:Saved figure at: ./img/global_step_150.png
INFO:root:global_step: 150, logpy: -52.591, kl: 13.062, loss: 63.993
 40%|████      | 200/500 [03:28<04:44,  1.05it/s]INFO:root:Saved figure at: ./img/global_step_200.png
INFO:root:global_step: 200, logpy: -34.038, kl: 19.023, loss: 52.057
 42%|████▏     | 208/500 [03:40<05:18,  1.09s/it]INFO:root:global_step: 208, logpy: -31.284, kl: 19.831, loss: 50.188


 50%|█████     | 250/500 [04:20<03:53,  1.07it/s]INFO:root:Saved figure at: ./img/global_step_250.png
INFO:root:global_step: 250, logpy: -18.701, kl: 23.910, loss: 42.004
 55%|█████▌    | 277/500 [04:51<03:28,  1.07it/s]INFO:root:global_step: 277, logpy: -11.965, kl: 26.181, loss: 37.683


 60%|██████    | 300/500 [05:13<03:07,  1.07it/s]INFO:root:Saved figure at: ./img/global_step_300.png
INFO:root:global_step: 300, logpy: -7.210, kl: 27.905, loss: 34.748
 69%|██████▉   | 347/500 [06:03<02:27,  1.04it/s]INFO:root:global_step: 347, logpy: 0.386, kl: 30.631, loss: 30.016


 70%|███████   | 350/500 [06:06<02:24,  1.03it/s]INFO:root:Saved figure at: ./img/global_step_350.png
INFO:root:global_step: 350, logpy: 0.801, kl: 30.769, loss: 29.746
 80%|████████  | 400/500 [06:58<01:33,  1.07it/s]INFO:root:Saved figure at: ./img/global_step_400.png
INFO:root:global_step: 400, logpy: 6.184, kl: 32.710, loss: 26.391
 83%|████████▎ | 417/500 [07:19<01:19,  1.04it/s]INFO:root:global_step: 417, logpy: 7.574, kl: 33.203, loss: 25.516


 90%|█████████ | 450/500 [07:50<00:46,  1.06it/s]INFO:root:Saved figure at: ./img/global_step_450.png
INFO:root:global_step: 450, logpy: 9.880, kl: 34.017, loss: 24.056
 97%|█████████▋| 487/500 [08:29<00:12,  1.06it/s]INFO:root:global_step: 487, logpy: 12.025, kl: 34.786, loss: 22.705


100%|██████████| 500/500 [08:42<00:00,  1.04s/it]
